In [394]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from matplotlib import pyplot as plt

import http
import requests
import yfinance as yf

In [395]:
api_key = "b5569108b07766dcb23028a5de53e385"

In [396]:
# This is how we get the series_id

# request_for_allReleases = f"https://api.stlouisfed.org/fred/series/search?search_text=volatility+index&api_key={api_key}&file_type=json"
# data = requests.get(request_for_allReleases).json()
# data

In [397]:
request_for_VIX = f"https://api.stlouisfed.org/fred/series/observations?series_id=VIXCLS&api_key={api_key}&file_type=json"
vix_data = requests.get(request_for_VIX).json()

request_for_VXV = f"https://api.stlouisfed.org/fred/series/observations?series_id=VXVCLS&api_key={api_key}&file_type=json"
vxv_data = requests.get(request_for_VXV).json()

In [398]:
vix_value = []
vix_date = []
for data in vix_data["observations"]:
    if (data["value"] != "."):
        vix_value.append(float(data["value"]))
        vix_date.append(data["date"])

vxv_value = []
vxv_date = []
for data in vxv_data["observations"]:
    if (data["value"] != "."):
        vxv_value.append(float(data["value"]))
        vxv_date.append(data["date"])


offset = len(vix_value) - len(vxv_value)

In [399]:
vix_value = vix_value[offset - 4:]
vix_date = vix_date[offset - 4:]

print(len(vix_date), len(vxv_date))
for i in range(len(vix_date)):
    if (i < len(vxv_date) and vix_date[i] != vxv_date[i]):
        vix_value.remove(vix_value[i])
        vix_date.remove(vix_date[i])

        i -= 1

vix_value = np.array(vix_value)
vxv_value = np.array(vxv_value)

3739 3735


In [400]:
vix_value.shape, vxv_value.shape

((3735,), (3735,))

In [401]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

# fig.add_trace(go.Candlestick(x=df.index,
#   open=df['Open'],
#   high=df['High'],
#   low=df['Low'],
#   close=df['Close']), row=1, col=1)

fig.add_trace(go.Scatter(x=vix_date, 
                         y=vix_value, 
                         line=dict(color='green', width=2), 
                         name='VIX'), row=1, col=1)
fig.add_trace(go.Scatter(x=vix_date, 
                         y=vxv_value, 
                         line=dict(color='red', width=2), 
                         name='VXV'), row=2, col=1)
fig.add_trace(go.Scatter(x=vix_date, 
                         y=vix_value, 
                         line=dict(color='green', width=2), 
                         name='VIX'), row=3, col=1)
fig.add_trace(go.Scatter(x=vix_date, 
                         y=vxv_value, 
                         line=dict(color='red', width=2), 
                         name='VXV'), row=3, col=1)
fig.show()

In [402]:
signal = vix_value - vxv_value
signal

array([-0.86, -0.8 , -1.14, ..., -0.24, -0.59, -0.6 ])

In [403]:
fig.add_trace(go.Scatter(x=vix_date, 
                         y=signal, 
                         line=dict(color='red', width=2), 
                         name='signal'), row=3, col=1)
fig.show()

In [404]:
vxv_date[0], vxv_date[-1]

('2007-12-04', '2022-10-04')

In [405]:
symbol = '^GSPC'
df = yf.download(symbol, start="2007-12-05", end="2022-10-05")
SP500 = df["Close"].to_numpy()

[*********************100%***********************]  1 of 1 completed


In [406]:
symbol = 'IEF'
df = yf.download(symbol, start="2007-12-05", end="2022-10-05")
IEF = df["Adj Close"].to_numpy()

[*********************100%***********************]  1 of 1 completed


In [407]:
print("vix value:", vix_value.shape)
print("vxv value:", vxv_value.shape)
print("vix date:", len(vix_date))
print("vxv date:", len(vxv_date))
print("S&P500:", SP500.shape)
print("IEF:", IEF.shape)

vix value: (3735,)
vxv value: (3735,)
vix date: 3735
vxv date: 3735
S&P500: (3735,)
IEF: (3735,)


In [408]:
date = df.index.to_numpy()

In [409]:
new_signal = signal > 0
new_signal = np.diff(new_signal.astype(int))
new_signal = np.insert(new_signal, 0, 0.0, axis = 0)
new_signal

array([ 0,  0,  0, ..., -1,  0,  0])

In [410]:
init_money = 1000
init_storage = 0.0
fee_rate = 0.01

In [411]:
def trade(money, storage, side, price):

    if side == -1:
        storage += money / (price * (1 + fee_rate))
        money -= money
    elif side == 1:
        money += storage * price * (1 - fee_rate)
        storage -= storage
    return money, storage


In [412]:
window_size = 250
off = 250
start = 0

profit = []
while start + window_size < vix_value.shape[0]:
    # print(start)
    money = init_money
    storage = init_storage

    local_signal = new_signal[start:start + window_size] != 0
    local_SP500 = SP500[start:start + window_size][local_signal]
    local_signal = (new_signal[start:start + window_size])[local_signal]

    trade(money, storage, -1, SP500[start])
    # print("trade:", 0, "M:", money, "S:", storage)

    # local_IEF = IEF[start:start + window_size][local_signal]


    for i in range(len(local_signal)):
        money, storage = trade(money, storage, local_signal[i], local_SP500[i])
        # print("trade:", i, "side", local_signal[i], "M:", money, "S:", storage)

    profit.append((money + storage * SP500[start + window_size - 1] - init_money) / init_money)

    start += off

In [426]:
window_size = 250
off = 250
start = 0

year_index = 1
while start + window_size < vix_value.shape[0]:
    profit = []
    money = init_money
    storage = init_storage
    previous_money = init_money

    # local_signal = new_signal[start:start + window_size] != 0
    # local_SP500 = SP500[start:start + window_size][local_signal]
    # local_signal = (new_signal[start:start + window_size])[local_signal]

    trade(money, storage, -1, SP500[start])
    
    for i in range(len(new_signal)):
        money, storage = trade(money, storage, new_signal[i], SP500[i])
        profit.append((money + storage * SP500[i] - previous_money) / previous_money)
        previous_money = money + storage * SP500[i]

    profit = np.array(profit)
    # profit = np.array(profit) + 1
    # profit = np.sign(profit) * np.abs(profit)**(365/window_size) - 1
    print("*---------------------------------------*")
    print("Year:", year_index)
    print(profit)
    print("expected return:", profit.mean(), "standard deviation:", profit.std())
    print("sharpe:", (profit.mean() - 0.02) / profit.std())
    year_index += 1
    start += off

*---------------------------------------*
Year: 1
[ 0.          0.          0.         ... -0.00990099  0.02588389
  0.0305837 ]
expected return: 0.0003775579698239316 standard deviation: 0.009112640583791914
sharpe: -2.1533211860761066
*---------------------------------------*
Year: 2
[ 0.          0.          0.         ... -0.00990099  0.02588389
  0.0305837 ]
expected return: 0.0003775579698239316 standard deviation: 0.009112640583791914
sharpe: -2.1533211860761066
*---------------------------------------*
Year: 3
[ 0.          0.          0.         ... -0.00990099  0.02588389
  0.0305837 ]
expected return: 0.0003775579698239316 standard deviation: 0.009112640583791914
sharpe: -2.1533211860761066
*---------------------------------------*
Year: 4
[ 0.          0.          0.         ... -0.00990099  0.02588389
  0.0305837 ]
expected return: 0.0003775579698239316 standard deviation: 0.009112640583791914
sharpe: -2.1533211860761066
*---------------------------------------*
Year: 5
[ 

In [414]:
profit = np.array(profit) + 1
profit = np.sign(profit) * np.abs(profit)**(365/window_size) - 1
print(len(profit))

21


In [415]:
print("expected return:", profit.mean(), "standard deviation:", profit.std())

expected return: 1.2120477705747523 standard deviation: 2.101954194364455


In [416]:
print("sharpe:", (profit.mean() - 0.02) / profit.std())

sharpe: 0.5671140568956018


In [417]:
inside_profit = []

money = init_money
storage = init_storage

previous_money = init_money
trade(money, storage, -1, SP500[start])

for i in range(len(new_signal)):
    money, storage = trade(money, storage, new_signal[i], SP500[i])
    # print("trade:", i, "side", local_signal[i], "M:", money, "S:", storage)

    pro = (money + storage * SP500[i] - init_money) / init_money
    pro_year = (1 + pro) ** (365 / (i + 1)) - 1
    inside_profit.append(pro_year)

